# 04. Model Training

In [ ]:
# 01. APRENDIZADO DE MÁQUINA (CLASSIFICAÇÃO PREDITIVA)

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import warnings

warnings.filterwarnings("ignore")

print("="*80)
print("Iniciando avaliação do Modelo Preditivo (Comparativo de Estímulos)")
print("="*80)

# 1. Carregamento e Isolamento da Tarefa
caminho_csv = '../reports/tabela_features_eeg_completa.csv'
df = pd.read_csv(caminho_csv)

SEED = 97
cv_strategy = LeaveOneOut() # LOOCV

# Dicionário para iterar pelas três condições
condicoes_busca = {
    'Face Feliz': ['Face Feliz', 'FF', 'F'],
    'Face Neutra': ['Face Neutra', 'FN', 'N'],
    'Face Raiva': ['Face Raiva', 'FR', 'R']
}

# 2. Estruturação do Pipeline de Aprendizado de Máquina
pipeline_classificacao = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler()),
    ('selector', SelectKBest(score_func=f_classif, k=15)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=3, random_state=SEED))
])

# 3. Validação e Extração de Métricas
for nome_condicao, lista_triggers in condicoes_busca.items():
    print(f"\n" + "-"*60)
    print(f"ESTÍMULO ANALISADO: {nome_condicao.upper()}")
    print("-" * 60)
    
    df_f = df[df['Condicao'].isin(lista_triggers)].copy()
    
    if df_f.empty:
        print(f"Atenção: Sem dados para {nome_condicao}.")
        continue

    # Garantia de integridade amostral (N=42)
    df_f = df_f.groupby(['ID', 'Grupo']).mean(numeric_only=True).reset_index()

    print(f"Amostra consolidada: {df_f.shape[0]} sujeitos.\n")

    y = df_f['Grupo'].apply(lambda x: 1 if 'TEA' in x else 0).values
    # Removemos colunas não numéricas antes de enviar ao modelo
    X = df_f.drop(columns=['ID', 'Grupo', 'Condicao', 'Tipo'], errors='ignore')

    y_pred = cross_val_predict(pipeline_classificacao, X, y, cv=cv_strategy)

    acc = accuracy_score(y, y_pred)
    cm = confusion_matrix(y, y_pred)

    print(f"Acurácia Global do Sistema: {acc:.2%}\n")

    print("Matriz de Confusão:")
    print(f"                   Predito Controle (0) | Predito TEA (1)")
    print(f"Real Controle (0) |        {cm[0,0]:02d}           |        {cm[0,1]:02d}")
    print(f"Real TEA (1)      |        {cm[1,0]:02d}           |        {cm[1,1]:02d}\n")

    print("Relatório de Desempenho (Precision, Recall, F1-Score):")
    report = classification_report(y, y_pred, target_names=['Controle', 'TEA'], digits=3)
    print(report)

print("="*80)

Iniciando avaliação do Modelo Preditivo (Comparativo de Estímulos)

------------------------------------------------------------
ESTÍMULO ANALISADO: FACE FELIZ
------------------------------------------------------------
Amostra consolidada: 42 sujeitos.

Acurácia Global do Sistema: 76.19%

Matriz de Confusão:
                   Predito Controle (0) | Predito TEA (1)
Real Controle (0) |        19           |        05
Real TEA (1)      |        05           |        13

Relatório de Desempenho (Precision, Recall, F1-Score):
              precision    recall  f1-score   support

    Controle      0.792     0.792     0.792        24
         TEA      0.722     0.722     0.722        18

    accuracy                          0.762        42
   macro avg      0.757     0.757     0.757        42
weighted avg      0.762     0.762     0.762        42


------------------------------------------------------------
ESTÍMULO ANALISADO: FACE NEUTRA
-------------------------------------------------